In [1]:
import os
import os
import pandas as pd
import numpy as np
os.environ['RNASTRUCTURE_PATH']="\exe"
os.environ['DATAPATH']="D:\Code\kaggle\RNA_Folding\RNA-FM-Transformer\RNAstructure\RNAstructure\data_tables"


In [2]:
import os
import pandas as pd
import numpy as np

def run_command(cmd):
    import subprocess
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()
    return output.decode('utf-8')

def predict_partition(self, temperature_k =None, dms = None):
    # predict the partition of rna structures
    cmd = f"{os.path.join(self.rnastructure_path, 'partition')} {self.fasta_file} {self.pfs_file}"
    if temperature_k != None:
        cmd += ' --temperature '+str(temperature_k)
    if type(dms) != type(None):
        assert len(self.sequence) == len(dms), 'The length of the sequence is not the same as the length of the signal.'
        assert type(dms) in [list, tuple, np.ndarray], 'The dms signal should be a list of floats.'
        self.__write_dms_to_file(self.sequence, dms)
        cmd += ' --dms ' + self.dms_file
    run_command(cmd)
    
    # sum it into pairing probability
    run_command(os.path.join(self.rnastructure_path,'ProbabilityPlot')+ ' ' +self.pfs_file + ' -t '+self.prob_file)
    
    # read the probability file 
    with open(self.prob_file,"r") as f:
        lines=f.readlines()
        pairingPrediction={'i':[],'j':[],'p':[]}
        for x in range(len(lines)):
            if x>1:
                ls=lines[x].split("\t")
                pairingPrediction["i"]+=[int(ls[0])]
                pairingPrediction["j"]+=[int(ls[1])]
                pairingPrediction["p"]+=[float(ls[2])]
    df = pd.DataFrame(pairingPrediction)
    
    return df

In [2]:
from RNAstructure import predictFromSequence
predict = predictFromSequence(
        'TTAAACCGGCCAACATACCGCATATGAGGATCACCCATATGCTCAAGATATTCGAAAGAATATCTTTCCACAGTCGAAAGACTGTGTCTCTCTCTTCCTTTTTCTCTTCCTCTTTCTCTTTCTCTTTCTCTTCTCTTCTGTATTACGAGTTCGCTACTCGTTCCTTTCGA',
        dms = np.random.random(170),
        predict_pairs= True,
        rnastructure_path='D:/Code/kaggle/RNA_Folding/RNA-FM-Transformer/RNAstructure/RNAstructure/exe',
        only_pair = True)

In [3]:
predict

[0.0013301438139521147,
 0.03798673973718806,
 0.2919107536612969,
 0.3118082541240563,
 0.23799481908885636,
 0.01222499984864899,
 0.18048977716406622,
 0.6921972088047781,
 0.6463238626449213,
 0.019177330161129446,
 0.001837687855771191,
 0.008717726263454607,
 0.009496762453460941,
 0.007404868132432124,
 0.12997156363506782,
 0.14561688404380593,
 0.15056870909153586,
 0.2653765337938382,
 0.10972608159620634,
 0.9738327084849513,
 0.9959972238323106,
 0.9957240704329282,
 0.9935574548385603,
 0.9961943689355613,
 0.9988862238032836,
 0.9989865431542411,
 0.0002089128841599848,
 0.995362182848953,
 0.9860692490995936,
 0.0001457857776196224,
 9.917528920621379e-06,
 4.610086056341273e-05,
 3.1457832095597775e-05,
 0.9852790077798792,
 0.9952861060704296,
 0.9991985935749037,
 0.9988467104930517,
 0.9961778587684282,
 0.9935496927824115,
 0.9957766184017353,
 0.9960352283154998,
 0.9220775413217099,
 0.001053420517402786,
 0.002793043259222992,
 0.9055416309014882,
 0.988656872601

In [7]:
len(predict['pairing']),len('TTAAACCGGCCAACATACCGCATATGAGGATCACCCATATGCTCAAGATATTCGAAAGAATATCTTTCCACAGTCGAAAGACTGTGTCTCTCTCTTCCTTTTTCTCTTCCTCTTTCTCTTTCTCTTTCTCTTCTCTTCTGTATTACGAGTTCGCTACTCGTTCCTTTCGA')

(170, 170)